# Modul Spezielle Anwendungen der Informatik: 
# K.I. in der Robotik

### Wintersemester 2018/2019 - HTW Berlin

### Rekurrente neuronale Netze in PyTorch am Beispiel eines simplen LSTM-Maschinenübersetzers

In [1]:
from IPython.display import Image
from IPython.core.display import HTML 

In [2]:
### Framework imports
import torch
from torch import optim
import os
import random

### Custom imports 
from model.model import *
from experiment.train_eval import evaluateInput, GreedySearchDecoder, trainIters, eval_batch, plot_training_results
from global_settings import device, FILENAME, SAVE_DIR, PREPRO_DIR, TRAIN_FILE, TEST_FILE, EXPERIMENT_DIR, LOG_FILE
from model.model import EncoderLSTM, DecoderLSTM
from utils.prepro import read_lines, preprocess_pipeline, load_cleaned_data, save_clean_data
from utils.tokenize import build_vocab, batch2TrainData, indexesFromSentence

from global_settings import DATA_DIR
from utils.utils import split_data, filter_pairs, max_length, plot_grad_flow

In [3]:
from translate import translate

### Projekt
- Rekurrente neuronale Netze, LSTMs
- Anwendung in PyTorch
- Implementierung eines naiven LSTM-Maschinenübersetzers, der in der Lage ist, kurze Sätze ("how are you" > "wie geht es dir") zu übersetzen
- Übersetzung: EN > DE
- Datensatz: [Tatoeba-Projekt](https://tatoeba.org/) (ca. 174000 Sprachpaare)
    - Reduzierung auf ca. 159.000 (Satzlänge 10)
    - Sprachdomäne: Alltag --> Ganz einfache Sätze

### Das Ergebnis

In [4]:
BEST_EXPERIMENT = "experiment/checkpoints/dry_run_simple_nmt_model_full_158544_teacher_1.0_train_voc_adam_lr-0.001-1/deu.txt/2-2_512-512_100"
SECOND_BEST_EXPERIMENT = ""

In [ ]:
print("Projektpresäntation...")
translate(start_root=".", path=BEST_EXPERIMENT)

Projektpresäntation...
Reading experiment information from: 
Starting translation process...
Source > hi
Translation >  hallo
Source > how are you
Translation >  wie gehts dir
Source > i am fine
Translation >  es geht mir gut
Source > i am in the university
Translation >  ich bin in der universitaet
Source > I am sad
Translation >  ich bin traurig
Source > I am happy
Translation >  ich bin gluecklich
Source > I love pizza
Translation >  ich liebe pizza
Source > I really enjoy films
Translation >  mir gefaellt mir sehr gerne
Source > I really enjoy football
Translation >  mir gefaellt mir sehr gerne football
Source > I like reading books
Translation >  ich lese gerne buecher
Source > I am in the kitchen with my mother
Translation >  ich bin in der kueche meiner mutter
Source > I am smart
Translation >  ich bin schlau
Source > I can translate short sentences
Translation >  ich kann das saetze uebersetzen
Source > the train has already gone
Translation >  der zug ist schon weg
Source > I 

## Modellarchitektur

### Sequence-to-Sequence (Seq2Seq) Modell oder Encoder-Decoder-Architektur

- Bezeichnung für einen neuronalen Maschinenübersetzer
- Verarbeitung von Sequenzen: Eingaben und Ausgaben sind Sequenzen, deren Länge im Voraus nicht bekannt ist. Diese Länge muss auch nicht übereinstimmen

<img src="documentation/seq2seq.png" alt="Seq2Seq Models" width=600>

Quelle: https://towardsdatascience.com/nlp-sequence-to-sequence-networks-part-2-seq2seq-model-encoderdecoder-model-6c22e29fd7e1

- **S** ist der sogenannte Kontextvektor, eine Zusammenfassung der verarbeiteten Sequenz
    - Sequenzen liegen als Zahlen vor und das Modell soll aus ihnen lernen
    - Insbesondere muss das Modell die "Semantik" lernen
        - Große Schwierigkeiten (!)
        - Ferne Zusammenhänge: **Der Mann**, der an der Haltestelle steht, **ist** mein Onkel --> Zwischen Subjekt und Verb stehen 5 Wörter! Für Menschen ist das easy, für Maschinen etwas weniger

### Das Geheimnis ist das Gedächtnis --> Rekurrente neuronale Netze (RNNs)

- Informationen "persistieren"
- Eingaben/Ausgaben variabler Länge verarbeiten
- Parameter werden durch die ganze Sequenz geteilt --> Sequenz (z.B. maximale Satzlänge in einer Batch) == Schritte == Zeit
    - Parameter:
        - U --> unter Eingaben und hidden layers geteilt
        - W --> unter den hidden layers geteilt
        - V --> unter hidden layers und Ausgaben geteilt

<img src="documentation/rnn.jpg" alt="Seq2Seq Models" width=600>

Quelle: http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/

Bezeichnungen:
- $x_{t}$ - Input beim Schritt $t$, z.B. das zweite Wort
- $s_{t}$ ist der Hidden-State zum Schritt $t$. Das ist der Speicher vom ganzen Netz: $s_{t} = \text{f}(Ux_t + Ws_{t-1})$ (f = tanh oder ReLU)
- $o_{t}$ ist der Ausgabe == Wahrscheinlichkeitsverteilung über alle möglichen Klassen, daher: $o_{t} = softmax(Vs_{t})$

---> "Short Memory"

### LSTMs (Long Short-Term Memory)

- Hidden State (h oder s) + Cell State (c) (Memory)

<img src="documentation/LSTM3-chain.png" alt="Seq2Seq Models" width=600>
Quelle: https://colah.github.io/posts/2015-08-Understanding-LSTMs/

Jede Zelle im Bild ist ein Neuron im ganzen Netz.

**Gating-Mechanismus**:

<img src="documentation/lstm_medium.png" alt="Seq2Seq Models" height=500 width=650>
Quelle: https://medium.com/@saurabh.rathor092/simple-rnn-vs-gru-vs-lstm-difference-lies-in-more-flexible-control-5f33e07b1e57

- **Forget**-Gate --> Wie viel vom "Gedächtnis" muss behalten werden? $f_t$
- **Input**-Gate -->  $ i_t$
- Berechnung der C-Kandidaten: $\tilde C_{t}$
- **Output-Gate**: $o_t$

### Pipeline
1. Vocabularies erstellen
2. Wörter darstellen --> Embeddings
3. Batch generieren --> (seq_len, batch_size)
4. Training:
    - Batch wird vom Encoder (als Batch) verarbeitet. Encoder liefert h
    - Decoder wird durch SOS initialisiert + Target-Batch + encoder hidden state (als erstes hidden state)
    - Bei jedem Schritt wird ein Wort vorhergesagt + hidden state aktualisiert
5. Die Übersetzung besteht aus den Wörtern mit der höchsten Wahrscheinlichkeit 

#### Einige Maßnahmen:
- Backpropagation Through Time --> Rekursion wird berücksichtigt, daher "Through Time"
- Teacher Forcing gegen langsame Konvergenz bzw. schlechte Leistungen
- Gradient Clipping --> kein Vanishing oder Exploding Gradient

In [ ]:
### Data cleaning
start_root = "."
exp_contraction = True # don't --> do not
file_to_load = "simple_dataset_praesi.txt"
file_name = "simple_dataset_praesi.pkl"


if os.path.isfile(os.path.join(start_root, PREPRO_DIR,file_name)):
    ##load
    print("File exists. Loading cleaned pairs...")
    pairs = load_cleaned_data(PREPRO_DIR, filename=file_name)
else: 
    print("Preprocessing file...")
    ### read lines from file
    pairs = read_lines(os.path.join(start_root,DATA_DIR),file_to_load)
    ### Preprocess file
    pairs, path = preprocess_pipeline(pairs, file_name, exp_contraction, max_len = 0)
    
print(random.choice(pairs))
print(random.choice(pairs))
print(random.choice(pairs))
print(random.choice(pairs))

In [ ]:
train_pairs = pairs
src_sents = [pair[0] for pair in pairs]
trg_sents = [pair[1] for pair in pairs]

max_src_l = max_length(src_sents)
max_trg_l = max_length(trg_sents)

print("Max length in source sentences:", max_src_l)
print("Max length in target sentences:", max_trg_l)

#### Vocabularies Example

In [ ]:
### Creating vocabularies
input_lang = build_vocab(src_sents, "eng")
output_lang = build_vocab(trg_sents, "deu")

print("Total source words:", input_lang.num_words)
print("Total target words:", output_lang.num_words)
print("")
print(input_lang.word2index)
print("")
#print(output_lang.word2index)
#print("")
print("Example of conversion word > index:")
print("Word {} > Index {}".format('hello', input_lang.word2index.get('hello')))
print("Index {} > Word {}".format(20, input_lang.index2word.get(20)))

#### Batching Example

<img src="documentation/seq2seq_batches.png">

In [ ]:
### Simple conversion sentence to tensor:
random_pair = train_pairs[40]
print(random_pair)

In [ ]:
english_sent = indexesFromSentence(input_lang, random_pair[0])
german_sent = indexesFromSentence(output_lang, random_pair[1])

print(english_sent)
print(german_sent)

In [ ]:
### No splitting for this short presentation :-)
train_pairs = pairs
mini_batch = 5
batch_pair = [random.choice(train_pairs) for _ in range(5)]
test_batch = batch_pair
test_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
for pair in test_batch:
    print("Source:", pair[0],"Target:", pair[1])    
    print("Src tensor:", indexesFromSentence(input_lang, pair[0]),"Trg tensor:", indexesFromSentence(output_lang, pair[1]))    

In [ ]:
### Creating a simple batch of 5 sentences --> Shape (seq_len, batch_size)
training_batch = batch2TrainData(input_lang, output_lang, batch_pair)

In [ ]:
input_tensor, input_lengths, target_tensor, mask, target_max_len, target_lengths = training_batch

In [ ]:
print("Length of source sentences:", input_lengths)

In [ ]:
print("Tensorized input:")
print(input_tensor)

In [ ]:
print("Tensorized output:")
print(target_tensor)
print(mask)

### Encoder

In [ ]:
### Das sieht der Encoder...
for i, elem in enumerate(input_tensor):
    print("Timestep:", i)
    print("Input:", elem)
    print("Woerter:", [input_lang.index2word[word.item()] for word in elem])


# Quellen

### Praxis: 

- Chatbot-Tutorial (PyTorch): https://pytorch.org/tutorials/beginner/chatbot_tutorial.html
- Sequence-to-Sequence Tutorial (PyTorch): https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html
- Machine Learning Mastery "How to develop a neural machine translator from scratch": https://machinelearningmastery.com/develop-neural-machine-translation-system-keras/

### Theorie (eine Auswahl):
- Sutskever et al. (2014), "Sequence to Sequence Learning with Neural Networks": https://arxiv.org/abs/1409.3215
- Stanford NLP (CS224N): http://web.stanford.edu/class/cs224n/index.html#schedule

